In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

import requests
import json

import pybaseball
from pybaseball import statcast
pybaseball.cache.enable()

import os
import glob
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


import sys
import time

import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np
import pandas as pd
from IPython.display import HTML

sys.path.append("code/.")

import mglearn
from IPython.display import display

# Classifiers and regressors
from sklearn.dummy import DummyClassifier, DummyRegressor

# Preprocessing and pipeline
from sklearn.impute import SimpleImputer

# train test split and cross validation
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
)
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

pd.set_option("display.max_colwidth", 200)

In [4]:
# the 'full_training_data.csv' is generated by Marcus
# remember to include it as a part of the pipeline in the final report 
df = pd.read_csv('data_2014_to_2024/full_training_data.csv')
df.shape

(16290, 1581)

In [ ]:
type(df.columns.to_list())

for i in df.columns.to_list():
    if 

list

In [10]:
df.columns[-100], df.columns[-10]

('single_batter6_home_10', 'bb_pitcher_home_10')

In [16]:
def calc_ops(ab,bb,hbp,single,double,triple,hr,sf):
    '''
    calculate on base plus slugging average of a single batter  
    to calculate the rolling average, 
    simply take in the TOTAL VALUE of each param (up to last 10 games)
    '''
    h = single+double+triple+hr

    if ab + bb + hbp + sf == 0:
        obp = 0  # If denominator is 0, set OBP to 0
    else:
        obp = (h + bb + hbp) / (ab + bb + hbp + sf)
    
    if ab == 0:
        slg = 0  # If denominator is 0, set SLG to 0
    else:
        slg = (single + 2 * double + 3 * triple + 4 * hr) / ab

    return obp + slg

def calc_era(er,ip):
    '''
    calculate earned run average of a single pitcher
    to calculate the rolling average, 
    simply take in the TOTAL VALUE of each param (up to last 10 games)
    '''
    if ip == 0: 
        return None
    return er/ip

# add more metrics if needed 

In [17]:
def add_ops_era(df):
    for team in ['away','home']:
        for game in range(1,11):
            for pos in ['batter','pitcher']:
                if pos == 'batter':
                    for order in range(1,10):
                        df[f'ops_{pos}{order}_{team}_{game}'] = df.apply(
                            lambda row: calc_ops(
                                row[f'ab_{pos}{order}_{team}_{game}'],
                                row[f'bb_{pos}{order}_{team}_{game}'],
                                row[f'hbp_{pos}{order}_{team}_{game}'],
                                row[f'single_{pos}{order}_{team}_{game}'],
                                row[f'double_{pos}{order}_{team}_{game}'],
                                row[f'triple_{pos}{order}_{team}_{game}'],
                                row[f'hr_{pos}{order}_{team}_{game}'],
                                row[f'sf_{pos}{order}_{team}_{game}']
                            ), axis = 1)
                else: 
                     df[f'era_{pos}_{team}_{game}'] = df.apply(
                         lambda row: calc_era(
                             row[f'er_{pos}_{team}_{game}'],
                             row[f'ip_{pos}_{team}_{game}'],
                         ), axis = 1)
    return df 

In [18]:
df_ops_era = add_ops_era(df)
df_ops_era.shape[1]

/var/folders/4f/yqk707nn61d0n9l9qmlg7f4w0000gn/T/ipykernel_4920/894371067.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'ops_{pos}{order}_{team}_{game}'] = df.apply(
/var/folders/4f/yqk707nn61d0n9l9qmlg7f4w0000gn/T/ipykernel_4920/894371067.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'ops_{pos}{order}_{team}_{game}'] = df.apply(
/var/folders/4f/yqk707nn61d0n9l9qmlg7f4w0000gn/T/ipykernel_4920/894371067.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

1781

In [22]:
df_ops_era_cleaned = df_ops_era[[col for col in df.columns if 'ops' in col or 'era' in col]]
df_ops_era_cleaned.shape[1]

200

In [23]:
df_ops_era_cleaned.head()

,ops_batter1_away_1,ops_batter2_away_1,ops_batter3_away_1,ops_batter4_away_1,ops_batter5_away_1,ops_batter6_away_1,ops_batter7_away_1,ops_batter8_away_1,ops_batter9_away_1,era_pitcher_away_1,...,ops_batter1_home_10,ops_batter2_home_10,ops_batter3_home_10,ops_batter4_home_10,ops_batter5_home_10,ops_batter6_home_10,ops_batter7_home_10,ops_batter8_home_10,ops_batter9_home_10,era_pitcher_home_10
0,0.40,1.200000,0.200000,0.40,0.50,0.833333,0.00,1.50,0.666667,0.60,...,0.500000,0.00,0.50,0.5,0.000000,0.25,0.50,0.333333,0.000000,0.800000
1,0.60,0.000000,2.000000,2.75,0.00,0.000000,0.75,0.00,0.500000,0.25,...,0.666667,0.25,1.25,0.0,0.000000,1.25,0.00,1.250000,1.750000,0.176367
2,0.25,0.500000,0.500000,0.25,0.50,0.000000,0.00,0.00,0.000000,0.00,...,1.000000,0.40,1.55,0.8,0.900000,1.40,0.20,0.933333,0.000000,1.750000
3,0.40,0.833333,1.166667,2.50,1.25,1.250000,0.75,0.25,2.000000,0.00,...,2.000000,1.00,0.00,0.6,1.416667,0.00,0.00,1.500000,0.500000,0.000000
4,0.50,0.500000,0.000000,1.75,0.50,0.833333,0.50,0.00,1.000000,0.80,...,0.800000,0.45,0.45,1.6,1.600000,0.40,1.35,0.400000,2.666667,0.000000


In [25]:
df_ops_era_cleaned.to_csv('data_2014_to_2024/full_training_data_ops_era.csv')